In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
import pickle
import warnings

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('seaborn-v0_8')


In [2]:
train_data = pd.read_csv('nfl_train_data_2002_2020.csv')
test_data = pd.read_csv('nfl_test_data_2021_2024.csv')

print(f"Training data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")
print(f"Training period: {train_data['season'].min():.0f} - {train_data['season'].max():.0f}")
print(f"Test period: {test_data['season'].min():.0f} - {test_data['season'].max():.0f}")

Training data shape: (9706, 21)
Test data shape: (1624, 21)
Training period: 2002 - 2020
Test period: 2021 - 2023


In [3]:
# These are the feaures we determined to be important in feature engineering
X_features = [
    'off_1st_down', 'off_pass_yds', 'off_rush_yds', 'off_total_yds', 'off_turnovers',
    'def_1st_down', 'def_pass_yds', 'def_rush_yds', 'def_total_yds', 'def_turnovers', 
    'home', 'wins', 'losses', 'overtime'
]

y_target = 'outcome'


missing_train = [f for f in X_features if f not in train_data.columns]
missing_test = [f for f in X_features if f not in test_data.columns]

if missing_train:
    print(f"\nMissing features in training data: {missing_train}")
if missing_test:
    print(f"\nMissing features in test data: {missing_test}")

In [4]:
X_train = train_data[X_features].copy()
y_train = train_data[y_target].copy()
X_test = test_data[X_features].copy()
y_test = test_data[y_target].copy()

print("=== DATA PREPARATION SUMMARY ===")
print(f"Training features shape: {X_train.shape}")
print(f"Training target shape: {y_train.shape}")
print(f"Test features shape: {X_test.shape}")
print(f"Test target shape: {y_test.shape}")

print(f"\nMissing values in training features: {X_train.isnull().sum().sum()}")
print(f"Missing values in test features: {X_test.isnull().sum().sum()}")

=== DATA PREPARATION SUMMARY ===
Training features shape: (9706, 14)
Training target shape: (9706,)
Test features shape: (1624, 14)
Test target shape: (1624,)

Missing values in training features: 0
Missing values in test features: 0


In [5]:

# We want can cut down features more using RFE
best_accuracy = 0
best_n_features = 14
best_features = X_features.copy()
results = []

for n_features in range(6, 15):  # Test 6-14 features
    # Use RFE with logistic regression
    rfe = RFE(
        estimator=LogisticRegression(solver='liblinear', C=1.0, random_state=42),
        n_features_to_select=n_features,
        step=1
    )
    
    X_train_rfe = rfe.fit_transform(X_train, y_train)
    X_test_rfe = rfe.transform(X_test)
    
    # This will train the model on a subset of features
    model_rfe = LogisticRegression(solver='liblinear', C=1.0, random_state=42)
    model_rfe.fit(X_train_rfe, y_train)
    
    train_acc = accuracy_score(y_train, model_rfe.predict(X_train_rfe))
    test_acc = accuracy_score(y_test, model_rfe.predict(X_test_rfe))
    
    selected_features = [X_features[i] for i in range(len(X_features)) if rfe.support_[i]]
    
    results.append({
        'n_features': n_features,
        'train_acc': train_acc,
        'test_acc': test_acc,
        'selected_features': selected_features,
        'model': model_rfe,
        'rfe': rfe
    })
    
    print(f"n={n_features:2d}: Train={train_acc:.4f}, Test={test_acc:.4f} - Top features: {selected_features[:3]}")
    
    if test_acc > best_accuracy:
        best_accuracy = test_acc
        best_n_features = n_features
        best_features = selected_features

print(f"\nBEST STEPWISE MODEL: {best_n_features} features, {best_accuracy:.4f} ({best_accuracy*100:.2f}%) accuracy")

best_result = [r for r in results if r['n_features'] == best_n_features][0]
print(f"\n=== OPTIMAL FEATURE SET ({best_n_features} features) ===")
for i, feature in enumerate(best_result['selected_features'], 1):
    coef = best_result['model'].coef_[0][i-1]
    print(f"  {i:2d}. {feature:15s} (coef: {coef:7.4f})")

n= 6: Train=0.7922, Test=0.7950 - Top features: ['off_1st_down', 'off_turnovers', 'def_1st_down']
n= 7: Train=0.7935, Test=0.7937 - Top features: ['off_1st_down', 'off_turnovers', 'def_1st_down']
n= 8: Train=0.7935, Test=0.7937 - Top features: ['off_1st_down', 'off_turnovers', 'def_1st_down']
n= 9: Train=0.8070, Test=0.7943 - Top features: ['off_1st_down', 'off_rush_yds', 'off_turnovers']
n=10: Train=0.8153, Test=0.7937 - Top features: ['off_1st_down', 'off_rush_yds', 'off_turnovers']
n=11: Train=0.8159, Test=0.7956 - Top features: ['off_1st_down', 'off_rush_yds', 'off_turnovers']
n=12: Train=0.8199, Test=0.7962 - Top features: ['off_1st_down', 'off_rush_yds', 'off_total_yds']
n=13: Train=0.8204, Test=0.7962 - Top features: ['off_1st_down', 'off_pass_yds', 'off_rush_yds']
n=14: Train=0.8197, Test=0.7962 - Top features: ['off_1st_down', 'off_pass_yds', 'off_rush_yds']

BEST STEPWISE MODEL: 12 features, 0.7962 (79.62%) accuracy

=== OPTIMAL FEATURE SET (12 features) ===
   1. off_1st_dow

In [7]:
final_model = best_result['model']
final_features = best_result['selected_features']
final_rfe = best_result['rfe']

model_filename = 'nfl_logistic_model_v1.pkl'
with open(model_filename, 'wb') as f:
    pickle.dump({
        'model': final_model,
        'features': final_features,
        'rfe_selector': final_rfe,
        'feature_order': X_features,
        'performance': {
            'train_accuracy': best_result['train_acc'],
            'test_accuracy': best_result['test_acc'],
            'n_features': best_result['n_features']
        }
    }, f)

print(f"Model saved as: {model_filename}")

# Detailed coefficient analysis to show important features
coef_analysis = pd.DataFrame({
    'Variable': final_features,
    'Coefficient': final_model.coef_[0],
    'Abs_Coefficient': np.abs(final_model.coef_[0]),
    'Impact': ['High' if abs(c) > 0.5 else 'Medium' if abs(c) > 0.05 else 'Low' 
              for c in final_model.coef_[0]]
}).sort_values('Abs_Coefficient', ascending=False)

print(coef_analysis.to_string(index=False, float_format='%.4f'))

print(f"Final accuracy: {best_result['test_acc']:.1%}")


Model saved as: nfl_logistic_model_v1.pkl
     Variable  Coefficient  Abs_Coefficient Impact
off_turnovers      -0.8718           0.8718   High
def_turnovers       0.8667           0.8667   High
         home       0.5797           0.5797   High
       losses      -0.0945           0.0945 Medium
         wins       0.0943           0.0943 Medium
 def_1st_down      -0.0508           0.0508 Medium
 off_1st_down       0.0417           0.0417    Low
     overtime       0.0208           0.0208    Low
 off_rush_yds       0.0100           0.0100    Low
 def_rush_yds      -0.0097           0.0097    Low
def_total_yds      -0.0052           0.0052    Low
off_total_yds       0.0052           0.0052    Low
Final accuracy: 79.6%


In [8]:
# Now we want to train the final model on the entire dset 
complete_data = pd.read_csv('nfl_game_features_2002_2024.csv')

print(f"Complete dataset shape: {complete_data.shape}")
print(f"Period covered: {complete_data['season'].min():.0f} - {complete_data['season'].max():.0f}")
print(f"Total games: {complete_data.shape[0]:,}")

X_complete = complete_data[final_features].copy()  # Use optimal features from stepwise selection
y_complete = complete_data[y_target].copy()

print(f"\nFeature matrix shape: {X_complete.shape}")
print(f"Target vector shape: {y_complete.shape}")
print(f"Missing values: {X_complete.isnull().sum().sum()}")
print(f"Win rate: {y_complete.mean():.3f}")

Complete dataset shape: (11330, 21)
Period covered: 2002 - 2023
Total games: 11,330

Feature matrix shape: (11330, 12)
Target vector shape: (11330,)
Missing values: 0
Win rate: 0.500


In [9]:

# Use the same configuration as the optimal stepwise model
production_model = LogisticRegression(
    solver='liblinear', 
    C=1.0, 
    random_state=42,
    max_iter=1000
)

# Train on complete dataset with optimal features
production_model.fit(X_complete, y_complete)

# Calculate training accuracy on complete dataset
y_complete_pred = production_model.predict(X_complete)
complete_accuracy = accuracy_score(y_complete, y_complete_pred)

print(f"Training accuracy on complete dataset: {complete_accuracy:.4f} ({complete_accuracy*100:.2f}%)")


print(f"Stepwise model (validation): {best_result['test_acc']:.4f} ({best_result['test_acc']*100:.2f}%)")
print(f"Production model (complete):  {complete_accuracy:.4f} ({complete_accuracy*100:.2f}%)")

print(f"\n=== PRODUCTION MODEL COEFFICIENTS ===")
production_coef_analysis = pd.DataFrame({
    'Variable': final_features,
    'Coefficient': production_model.coef_[0],
    'Abs_Coefficient': np.abs(production_model.coef_[0])
}).sort_values('Abs_Coefficient', ascending=False)

print(production_coef_analysis.to_string(index=False, float_format='%.4f'))


Training accuracy on complete dataset: 0.8160 (81.60%)
Stepwise model (validation): 0.7962 (79.62%)
Production model (complete):  0.8160 (81.60%)

=== PRODUCTION MODEL COEFFICIENTS ===
     Variable  Coefficient  Abs_Coefficient
off_turnovers      -0.8862           0.8862
def_turnovers       0.8754           0.8754
         home       0.5197           0.5197
       losses      -0.0934           0.0934
         wins       0.0902           0.0902
 def_1st_down      -0.0532           0.0532
 off_1st_down       0.0431           0.0431
 off_rush_yds       0.0092           0.0092
 def_rush_yds      -0.0089           0.0089
     overtime       0.0084           0.0084
def_total_yds      -0.0058           0.0058
off_total_yds       0.0056           0.0056


In [10]:
production_model_filename = 'nfl_production_model_2002_2024.pkl'
with open(production_model_filename, 'wb') as f:
    pickle.dump({
        'model': production_model,
        'features': final_features,
        'feature_order': X_features,
        'training_period': f"{complete_data['season'].min():.0f}-{complete_data['season'].max():.0f}",
        'performance': {
            'training_accuracy': complete_accuracy,
            'training_samples': X_complete.shape[0],
            'n_features': len(final_features)
        },
        'coefficients': {
            'variables': final_features,
            'values': production_model.coef_[0].tolist()
        },
        'metadata': {
            'trained_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
            'solver': production_model.solver,
            'regularization': production_model.C,
            'random_state': production_model.random_state
        }
    }, f)
